# Planering av examination
## Frågeställning
- Vilka förutsättningar behöver ett dataset ha för att kunna träna en modell att prediktera om en sida är en förstasida eller inte i en text och avgöra var textelementen rubrik, text, bild, tabell och list?
- Vilken av följande modeller ger högst score och flest korrekta prediktioner i dessa?

## Källor
https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html - Exempel på hur man kan arbeta med texter från scikit learn
https://www.kaggle.com/datasets/devashishprasad/documnet-layout-recognition-dataset-publaynet-t0/data - Ett dataset med dokument som är annoterade med title, text, figure, table och list
https://guillaumejaume.github.io/FUNSD/description/ - Ett dataset med inskannade dokument för dokumentklassificering
https://labelformat.com/formats/object-detection/coco/ - Förklaring till COCO dataset format. Även: https://cocodataset.org/#format-data 

Använt för tips och idéer:
https://github.com/AyanGadpal/Document-Image-Augmentation
https://github.com/DevashishPrasad/CascadeTabNet

## Arbetsordning
1. Hantera data så att den går att använda
    - PLN:
        - Json-filen med annotationerna var väldigt stor så datorn klarade inte av att läsa in den rakt upp och ned istället fick jag skriva ett script som skulle hantera filen genom att läsa in delar med *ijson* och jag använde *polars* istället för pandas. Detta för att lättare kunna spara ned datan med *parquet* (ge lite grund till både polars och parquet). Eftersom 'annotations' var stor fick jag ta den i chunks.
        - Det visade sig också att json-filen innehöll samtliga annotationer för hela original-datasetet, men antalet dokument som fanns tillgängligt på kaggle.com innehöll knappt 48 000 dokument. Därför filtrerade jag DataFrame till att innehålla bara de rader som jag hade dokument för.
        - Förbereda dataset till binär klassficering av title_page True/False genom att lägga till en kolumn som står för om en bild är title_paage eller inte. Samtliga filer med '00000' är title page. Tog bort alla kolumner förutom title_page och file_name och behöll sedan en rad per unikt filnamn. Sedan checkade jag så att alla filnamn stämde överens mellan DF och /documents.
        - Förberedde bilderna genom att sätta gråskala, resize, normalisera pixelvärdena, flatten. Eftersom det var så många bilder körde jag dem i batcher.
        - Skapar ett mindre dataset för att utforska hur jag ska bygga en pipeline.
        - Utforskar hur RandomForest, SVM, LogisticRegression och KNeighborsClassifier beter sig på det mindre datasetet, genom att sätta upp olika hyperparametrar och testar genom GridSearchCV.
        - Utvärderar modellerna med den skapade funktionen evaluate_model() som inkluderar de grundläggande utvärderingmoåtten.
        - Hur ska jag tänka eftersom annotationerna är utifrån original pixelstorlek.
        - Hur ska jag kunna använda ord som keywords för att hitta relevanta texter? ÄR det lönt eller meningslöst. Bara databassök?
        - Plocka bort rad 41-128 för de är inte intressanta för första sidan. Lägga i pipeline för att engineer data innan träning.
        - Testar träning på förändrad data på både RFC och KNC samt vilka observationer som de predikterar fel eller rätt för att se om det är intressant att göra en ensemble, ex voting. Sammanfattningsvis ger RFC bra resultat vid threshold 0.2. Några fler FP men få FN. Ensemble med att ta ut de positiva prediktionerna från både KNC och RFC ger dock samma TP men färre FP
        - Skapar ett skript för utifrån ensemble för att testa på hela datasetet (dock konverterat till 128x128)
    - FUNSD:
        - 
Hur ska jag jobba med bilder?
2. Förstå hur det fungerar med dokumentigenkänning, element i dokument, text
    Steg att ta:
        - Binärt klassificering av dokument: Är det en förstasida eller inte.
        - Klustringsproblem?: Vilka förstasidor är av samma kategori?
        - Vilka textelement är av samma typ?
3. Använda ett färdigt exempel med dataset från Scikit Learn


## COCO dataset format
Ett standardformat för objekt-detektion och segmentering, som består av tre huvudnycklar: Images, Categories och Annotations. I Images ges detaljer om bilden/dokumentet i form av file_name, height, width och id. Categories ger de olika kategorierna som finns i bilden utifrån id som i sin tur är kopplat till name, ex id: 1, name: "text". Det kan också finnas supercategory, alltså överkategori. I Annotations finns följande:
- segmentation, som ger en lista med polygon-koordinater, vilka definierar exakta pixelgränser för kategorin. 
- bbox, i formatet [x, y, width, height] och som ger minsta rektangel som omsluter objektet, mätt från övre vänstra hörnet.
- area, som visar segmentets area.
- iscrowd i formatet 0 eller 1, som visar om det är ett enskilt objekt eller en grupp av objekt.
- image_id, som är en referens till bilden, till vilken annotationen tillhör.
- category_id i heltal, som anger vilken kategori (Categories) som segmentet tillhör.
- id, som är ett unikt id för respektive annotation.

Sammanfattningsvis har en bild många annotationer, varje annotation tillhör en kategori, annotationen har exakta gränser samt en rektangel med areauträkning.

## Images

# Presentation
## Ville göra något med text: 
    - Avgöra om en sida är förstasidan eller inte
    - Avgöra var de olika textelementen rubrik, text, bild, tabell och lista

## Dataset
Pub Lay Net - Är ett av IBM skapat dataset med texter om 4-8 sidor. 
- 48 000 dokument (originalet innehåller över 300 000) i jpg-format
- json med annotationer utifrån COCO med över 3.000.000 rader och annotationer till dokument jag inte hade.


## Resultat på 128x128

### Med PCA
```code
Setting threshold to 0.5
Testing prediction
[[8239   38]
 [  94 1221]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8277
           1       0.97      0.93      0.95      1315

    accuracy                           0.99      9592
   macro avg       0.98      0.96      0.97      9592
weighted avg       0.99      0.99      0.99      9592

Setting threshold to 0.4
Testing prediction
[[8183   94]
 [  65 1250]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8277
           1       0.93      0.95      0.94      1315

    accuracy                           0.98      9592
   macro avg       0.96      0.97      0.97      9592
weighted avg       0.98      0.98      0.98      9592
```


### Utan PCA
```code
Setting threshold to 0.4
Testing prediction
[[8218   59]
 [  42 1273]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8277
           1       0.96      0.97      0.96      1315

    accuracy                           0.99      9592
   macro avg       0.98      0.98      0.98      9592
weighted avg       0.99      0.99      0.99      9592

Setting threshold to 0.5
Testing prediction
[[8255   22]
 [  69 1246]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8277
           1       0.98      0.95      0.96      1315

    accuracy                           0.99      9592
   macro avg       0.99      0.97      0.98      9592
weighted avg       0.99      0.99      0.99      9592
```